In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

x = np.linspace(-4,4, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1))=


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = np.random.normal(size=10000000)
plt.hist(data, bins=25)


In [ ]:
min(data), max(data)

In [ ]:
import numpy as np
normal_data = np.random.rand(1000)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(normal_data)


In [ ]:
hard_to_detect = list(np.random.uniform(1,2,size=int(0.005*1000))) + \
                  list(np.random.uniform(0,-1,size=int(0.005*1000)))

easy_to_detect = list(np.random.uniform(2,3,size=int(0.005*1000))) + \
                  list(np.random.uniform(-1,-2,size=int(0.005*1000)))


In [ ]:
total_data = list(normal_data) + hard_to_detect + easy_to_detect

import random
random.shuffle(total_data)

for datapoint in total_data:
  pass


In [ ]:
# Anomaly percentage for the quantile thresholder
expected_percentage_anomaly = 20/1020
expected_percentage_normal = 1 - expected_percentage_anomaly


In [ ]:
!pip install river
from river import anomaly

model = anomaly.QuantileThresholder(
    anomaly.OneClassSVM(),
    q=expected_percentage_normal
    )

for datapoint in total_data:
    model = model.learn_one({'x': datapoint})


In [ ]:
scores = []
for datapoint in total_data:
    scores.append(model.score_one({'x': datapoint}))


In [ ]:
import pandas as pd
results = pd.DataFrame({'data': total_data, 'score': scores})
results['actual_outlier'] = (results['data'] > 1 ) | (results['data'] < 0)

# there are 20 actual outliers
results['actual_outlier'].value_counts()


In [ ]:
# the algo detected 22 outliers
results['score'].value_counts()


In [ ]:
# in the 22 detected otuliuers, 10 are actual outliers, but 12 are not actually outliers
results.groupby('actual_outlier')['score'].sum()


In [ ]:
from river import anomaly

model2 = anomaly.QuantileThresholder(
    anomaly.HalfSpaceTrees(),
    q=expected_percentage_normal
    )

for datapoint in total_data:
    model2 = model2.learn_one({'x': datapoint})


scores2 = []
for datapoint in total_data:
    scores2.append(model2.score_one({'x': datapoint}))
    
import pandas as pd
results2 = pd.DataFrame({'data': total_data, 'score': scores2})
results2['actual_outlier'] = (results2['data'] > 1 ) | (results2['data'] < 0)

# there are 20 actual outliers
results2['actual_outlier'].value_counts()


In [ ]:
# the algo detected 29 outliers
results2['score'].value_counts()


In [ ]:
# the 29 detected outliers are not actually outliers
results2.groupby('actual_outlier')['score'].sum()
